In [ ]:
import torch
import torch.nn as nn
import cv2, os

In [ ]:
face_cascade=cv2.CascadeClassifier()

In [ ]:
#@ For Cropping face in image:
def crop_face(img):
  gray=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  faces=face_cascade.detectMultiScale(gray, 1.3, 5)
  if(len(faces)>0):
    for (x, y, w, h) in faces:
      img2=img[y:(y+h), x:(x+w), :]
    img2=cv2.resize(img2, (256, 256))
    return img2, True
  else:
    return img, False

In [ ]:
#@ Helper Function:
def _ConvLayer(in_features, out_features):
  return nn.Sequential(
      nn.Conv2d(in_features, out_features,
                kernel_size=5, stride=2, padding=2),
      nn.LeakyReLU(0.1, inplace=True)
  )

def _UpScale(in_features, out_features):
  return nn.Sequential(
      nn.ConvTranspose2d(in_features, out_features,
                         kernel_size=2, stride=2, padding=0),
      nn.LeakyReLU(0.1, inplace=True)
  )

class Reshape(nn.Module):
  def forward(self, input):
    output=input.view(-1, 1024, 4, 4)
    return output

In [ ]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()

    self.encoder=nn.Sequential(
        _ConvLayer(3, 128),
        _ConvLayer(128, 256),
        _ConvLayer(256, 512),
        _ConvLayer(512, 1024),
        nn.Flatten(),
        nn.Linear(1024*4*4, 1024),
        nn.Linear(1024, 1024*4*4),
        Reshape(),
        _UpScale(1024, 512)
    )

    self.decoder_A=nn.Sequential(
         _UpScale(512, 256),
         _UpScale(256, 128),
         _UpScale(128, 64),
         nn.Conv2d(64, 3, kernel_size=3, padding=1),
         nn.Sigmoid()
    )

    self.decoder_B=nn.Sequential(
         _UpScale(512, 256),
         _UpScale(256, 128),
         _UpScale(128, 64),
         nn.Conv2d(64, 3, kernel_size=3, padding=1),
         nn.Sigmoid()
    )

  def forward(self, x, select='A'):
    if select == 'A':
      out=self.encoder(x)
      out=self.decoder_A(out)
    else:
      out=self.encoder(x)
      out=self.decoder_B(out)
    return out

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
model=AutoEncoder()
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           9,728
         LeakyReLU-2          [-1, 128, 32, 32]               0
            Conv2d-3          [-1, 256, 16, 16]         819,456
         LeakyReLU-4          [-1, 256, 16, 16]               0
            Conv2d-5            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-6            [-1, 512, 8, 8]               0
            Conv2d-7           [-1, 1024, 4, 4]      13,108,224
         LeakyReLU-8           [-1, 1024, 4, 4]               0
           Flatten-9                [-1, 16384]               0
           Linear-10                 [-1, 1024]      16,778,240
           Linear-11                [-1, 16384]      16,793,600
          Reshape-12           [-1, 1024, 4, 4]               0
  ConvTranspose2d-13            [-1, 512, 8, 8]       2,097,664
        LeakyReLU-14            [-1, 51